In [15]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [34]:
# to reload
%dotenv

In [35]:
from google.cloud import bigquery

client = bigquery.Client()

In [60]:
p_sample = pd.read_excel("../projects/gain_tracker/tests/positions_sample.xlsx")

In [44]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price           float64
position_exit_date      datetime64[ns]
position_exit_price            float64
recognized_gain                float64
dtype: object

### change to Decimals

In [47]:
from decimal import Decimal

In [ ]:
p_sample.loc[:, "position_entry_price"] = p_sample["position_entry_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

p_sample.loc[:, "position_exit_price"] = p_sample["position_exit_price"].apply(
    lambda p: Decimal(str(round(p, 6))))

In [52]:
p_sample.tail()

,portfolio_id,position_id,external_id,id_type,position_entry_date,exchange,quantity,status,position_entry_price,position_exit_date,position_exit_price,recognized_gain
26,Roth IRA,26,TSLA,TICKER,2024-01-25,PCX,2,OPEN_LONG,180.4,NaT,NaN,NaN
27,IRA,27,DQ,TICKER,2024-01-25,PCX,20,OPEN_LONG,19.5,NaT,NaN,NaN
28,IRA,28,CVS,TICKER,2024-01-25,PCX,8,OPEN_LONG,71.99,NaT,NaN,NaN
29,Roth IRA,29,BLDE,TICKER,2024-01-30,PCX,160,OPEN_LONG,3.0,NaT,NaN,NaN
30,IRA,30,GPS,TICKER,2024-01-31,PCX,20,OPEN_LONG,18.9,NaT,NaN,NaN


In [53]:
p_sample.dtypes

portfolio_id                    object
position_id                      int64
external_id                     object
id_type                         object
position_entry_date     datetime64[ns]
exchange                        object
quantity                         int64
status                          object
position_entry_price            object
position_exit_date      datetime64[ns]
position_exit_price             object
recognized_gain                float64
dtype: object

In [54]:
job = client.load_table_from_dataframe(
    p_sample, "main-street-labs.gain_tracker_dev.positions_dec"
)

In [56]:
# Perform a query.
QUERY = (
    'SELECT position_id, position_entry_price, position_exit_price FROM `main-street-labs.gain_tracker_dev.positions_dec`'
)
p_dec_sample_dl = client.query_and_wait(QUERY).to_dataframe()  # API request

In [108]:
p_dec_sample_dl.tail()

,position_id,position_entry_price,position_exit_price
26,1,156.000,158.0000
27,0,210.130,470.0000
28,4,7.895,9.4000
29,5,7.895,9.2349
30,3,39.030,46.0700


# Use ETrade API

In [16]:
import pandas as pd
from datetime import date

import os

In [114]:
# consumer_key = os.getenv("ETRADE_SANDBOX_KEY")
# consumer_secret = os.getenv("ETRADE_SANDBOX_SECRET")
consumer_key = os.getenv("ETRADE_KEY")
consumer_secret = os.getenv("ETRADE_SECRET")

In [13]:
from rauth import OAuth1Service
import webbrowser

In [115]:
service = OAuth1Service(
              name = 'etrade',
              consumer_key = consumer_key,
              consumer_secret = consumer_secret,
              request_token_url = 'https://apisb.etrade.com/oauth/request_token',
              access_token_url = 'https://apisb.etrade.com/oauth/access_token',
              authorize_url = 'https://us.etrade.com/e/t/etws/authorize?key={}&token={}',
              base_url = 'https://apisb.etrade.com')


In [116]:
oauth_token, oauth_token_secret = service.get_request_token(params = 
                                  {'oauth_callback': 'oob', 
                                   'format': 'json'})

In [119]:
auth_url = service.authorize_url.format(consumer_key, oauth_token)

In [120]:
# Get verifier (direct input in console, still working on callback)
webbrowser.open(auth_url)
verifier = input('Please input the verifier: ')

Please input the verifier:  MI960


In [121]:
session = service.get_auth_session(oauth_token, oauth_token_secret, params = {'oauth_verifier': verifier})

In [125]:
# list_accounts_url = 'https://apisb.etrade.com/v1/accounts/list.json'
list_accounts_url = 'https://api.etrade.com/v1/accounts/list.json'

In [126]:
resp = session.get(list_accounts_url, params = {'format': 'json'})

In [127]:
# resp.text
resp.json()

{'AccountListResponse': {'Accounts': {'Account': [{'accountId': '153285977',
     'accountIdKey': 'lP_HmzzVyiLFftb7DXaEUw',
     'accountMode': 'CASH',
     'accountDesc': 'Individual Brokerage',
     'accountName': '',
     'accountType': 'INDIVIDUAL',
     'institutionType': 'BROKERAGE',
     'accountStatus': 'ACTIVE',
     'closedDate': 0,
     'shareWorksAccount': False,
     'fcManagedMssbClosedAccount': False},
    {'accountId': '227699084',
     'accountIdKey': 'E2ayfoKSzH1fFZs-7xV48A',
     'accountMode': 'CASH',
     'accountDesc': 'Roth IRA',
     'accountName': '',
     'accountType': 'ROTHIRA',
     'institutionType': 'BROKERAGE',
     'accountStatus': 'ACTIVE',
     'closedDate': 0,
     'shareWorksAccount': False,
     'fcManagedMssbClosedAccount': False},
    {'accountId': '892405345',
     'accountIdKey': 'SXPtmpM7IZIpymUjhjI-zg',
     'accountMode': 'CASH',
     'accountDesc': 'Traditional IRA',
     'accountName': '',
     'accountType': 'CONTRIBUTORY',
     'institut

In [27]:
accountIdKey = "xj1Dc18FTqWPqkEEVUr5rw"
instType = "BROKERAGE"

In [96]:
account_balances_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/balance.json"

In [75]:
account_balances_url

'https://apisb.etrade.com/v1/accounts/xj1Dc18FTqWPqkEEVUr5rw/balance'

In [76]:
resp = session.get(account_balances_url, params = {"instType":instType,"realNAV": False})

In [77]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><BalanceResponse><accountId>833535350</accountId><accountType>MARGIN</accountType><optionLevel>LEVEL_4</optionLevel><accountDescription></accountDescription><Cash><fundsForOpenOrdersCash>0.0</fundsForOpenOrdersCash><moneyMktBalance>0.0</moneyMktBalance></Cash><Computed><cashAvailableForInvestment>0.00</cashAvailableForInvestment><cashAvailableForWithdrawal>0.00</cashAvailableForWithdrawal><netCash>-740.450013</netCash><cashBalance>0.0</cashBalance><settledCashForInvestment>0.0</settledCashForInvestment><unSettledCashForInvestment>0.0</unSettledCashForInvestment><fundsWithheldFromPurchasePower>0.0</fundsWithheldFromPurchasePower><fundsWithheldFromWithdrawal>0.0</fundsWithheldFromWithdrawal><OpenCalls><cashCall>0.0</cashCall></OpenCalls><RealTimeValues><totalAccountValue>0.0</totalAccountValue><netMv>-454.22</netMv><netMvLong>0.0</netMvLong></RealTimeValues></Computed></BalanceResponse>'

In [28]:
view_portfolio_url = f"https://apisb.etrade.com/v1/accounts/{accountIdKey}/portfolio.json"

In [29]:
viewp_resp = session.get(view_portfolio_url)

In [30]:
import json

In [31]:
data = json.loads(viewp_resp.text)

In [102]:
viewp_resp.json() == data

True

In [104]:
'PortfolioResponse' in data

True

In [32]:
data

{'PortfolioResponse': {'AccountPortfolio': [{'accountId': '833597000',
    'Position': [{'positionId': 27005131,
      'symbolDescription': 'BR',
      'dateAcquired': -57600000,
      'pricePaid': 0,
      'commissions': 0,
      'otherFees': 0,
      'quantity': 10,
      'positionIndicator': 'TYPE2',
      'positionType': 'LONG',
      'daysGain': -2.7999,
      'daysGainPct': -1.3346,
      'marketValue': 207,
      'totalCost': 0,
      'totalGain': 207,
      'totalGainPct': 0,
      'pctOfPortfolio': 0.0018,
      'costPerShare': 0,
      'todayCommissions': 0,
      'todayFees': 0,
      'todayPricePaid': 0,
      'todayQuantity': 0,
      'lotsDetails': 'https://apisb.etrade.com/v1/accounts/E5Nd4LJBsEi_UyHm4Vio9g/portfolio/27005131',
      'quoteDetails': 'https://apisb.etrade.com/v1/market/quote/BR',
      'Product': {'symbol': 'BR',
       'securityType': 'EQ',
       'expiryYear': 0,
       'expiryMonth': 0,
       'expiryDay': 0,
       'strikePrice': 0,
       'productId'